# Inport

In [1]:
import torch
import numpy as np
from dataset.dataset import getDatasetAndLoader
from model import getOptNet
from pyhocon import ConfigFactory,HOCONConverter
import argparse
import trimesh
import os
import os.path as osp
from MCAcc import Seg3dLossless
import utils

/home/mas/anaconda3/envs/SelfRecon/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# parameters

In [2]:
data_root="/mnt/data/study/computer_vision/selfrecon/female-3-casual"

In [3]:
parser = argparse.ArgumentParser(description='neu video body rec')
parser.add_argument('--gpu-ids',default=[0],nargs='+',type=int,metavar='IDs',
					help='gpu ids') # metavar: Alternate display name for the argument as shown in help 
parser.add_argument('--conf',default="config.conf",metavar='M',
					help='config file')
parser.add_argument('--data',default=data_root,metavar='M',
					help='data root')
parser.add_argument('--model',default=None,metavar='M',
					help='pretrained scene model')
parser.add_argument('--model-rm-prefix',nargs='+',type=str,metavar='rm prefix', help='rm model prefix')
parser.add_argument('--sdf-model',default=None,metavar='M',
					help='substitute sdf model')
parser.add_argument('--save-folder',default="result",metavar='M',help='save folder')
args = parser.parse_args(args=[])


In [4]:
config=ConfigFactory.parse_file(args.conf)
if len(args.gpu_ids):
	device=torch.device(args.gpu_ids[0])
else:
	device=torch.device(0)
data_root=args.data
if args.save_folder is None:
	print('please set save-folder...')
	assert(False)

save_root=osp.join(data_root,args.save_folder)
debug_root=osp.join(save_root,'debug')
os.makedirs(save_root,exist_ok=True)
os.makedirs(debug_root,exist_ok=True)

In [5]:
#point render
resolutions={'coarse':
[
	(14+1, 20+1, 8+1),
	(28+1, 40+1, 16+1),
	(56+1, 80+1, 32+1),
	(112+1, 160+1, 64+1),
	(224+1, 320+1, 128+1),
],
'medium':
[
	(18+1, 24+1, 12+1),
	(36+1, 48+1, 24+1),
	(72+1, 96+1, 48+1),
	(144+1, 192+1, 96+1),
	(288+1, 384+1, 192+1),
],
'fine':
[
	(20+1, 26+1, 14+1),
	(40+1, 52+1, 28+1),
	(80+1, 104+1, 56+1),
	(160+1, 208+1, 112+1),
	(320+1, 416+1, 224+1),
]
}

resolutions_higher = [
	(32+1, 32+1, 32+1),
	(64+1, 64+1, 64+1),
	(128+1, 128+1, 128+1),
	(256+1, 256+1, 256+1),
	(512+1, 512+1, 512+1),
]

In [6]:
# save the config file
with open(osp.join(save_root,'config.conf'),'w') as ff:
	ff.write(HOCONConverter.convert(config,'hocon'))
condlen={'deformer':config.get_int('mlp_deformer.condlen'),'renderer':config.get_int('render_net.condlen')}
batch_size=config.get_int('train.coarse.point_render.batch_size')
dataset,dataloader=getDatasetAndLoader(data_root,condlen,batch_size,
						config.get_bool('train.shuffle'),config.get_int('train.num_workers'),
						config.get_bool('train.opt_pose'),config.get_bool('train.opt_trans'),config.get_config('train.opt_camera'))

scene data use female smpl


In [7]:
bmins=None
bmaxs=None

In [8]:
if config.get_int('train.initial_iters')<=0:
	use_initial_sdf=True
else:
	use_initial_sdf=False


In [10]:
# memo: at this line, GPU memory usage: 224MB/8GB

In [11]:
optNet,sdf_initialized=getOptNet(dataset,batch_size,bmins,bmaxs,resolutions['coarse'],device,config,use_initial_sdf)

/home/mas/anaconda3/envs/SelfRecon/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811806235/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


RuntimeError: CUDA out of memory. Tried to allocate 1.29 GiB (GPU 0; 7.80 GiB total capacity; 5.22 GiB already allocated; 530.44 MiB free; 5.23 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [35]:
optNet,dataloader=utils.set_hierarchical_config(config,'coarse',optNet,dataloader,resolutions['coarse'])

NameError: name 'optNet' is not defined